In [257]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [258]:
from IPython.display import display, HTML, Video
display(HTML("<style>.container { width:90% !important; }</style>"))

# Load data

In [259]:
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader

import sentencepiece

In [260]:
import os
import glob
import json
import regex

import tqdm.notebook as tqdm

import numpy as np
import pandas as pd

from ipywidgets import GridBox, Audio, HBox, VBox, Box, Label, Layout

import matplotlib.pyplot as plt
# import matplotlib_inline

%matplotlib inline
# matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [261]:
def sample_dataset(dataset, n=4):
    grid = []
    for idx in range(n):
        grid.append(
            VBox([
                Label('{0:d}, {1}, {2:.1f}'.format(idx, dataset['text'][idx], dataset.get('duration', dataset['audio_len'])[idx])),
                Audio.from_file(dataset['audio_path'][idx], autoplay=False, loop=False),
            ])
        )
    
    return HBox([VBox(grid[0::2]), VBox(grid[1::2])])

In [262]:
base_path = '/home/jupyter/mnt/datasets'

libri_speech_base_path = os.path.join(base_path, 'LibriSpeech_ds')
golos_base_path = os.path.join(base_path, 'golos')

In [209]:
from src.dataset import get_libri_speech_dataset, get_golos_dataset

In [210]:
libri_speech_dev = get_libri_speech_dataset(libri_speech_base_path, split='dev')

print('Loaded {0:d} objects'.format(len(libri_speech_dev['audio_path'])))

Loaded 1400 objects


In [211]:
# Load tokenizer model
sp_tokenizer = sentencepiece.SentencePieceProcessor(model_file='tokenizer.model')

In [212]:
from src.dataset import AudioDataset, collate_fn

In [213]:
libri_speech_dev_ds = AudioDataset(libri_speech_dev, sp_tokenizer, min_duration=1.36, max_duration=10.96)
libri_speech_dev_ds[0]

/usr/local/lib/python3.8/dist-packages/pandas/core/computation/expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


('/home/jupyter/mnt/datasets/LibriSpeech_ds/dev/audio/5397/2145/poemi_16_pushkin_0039.wav',
 tensor([-4.8218e-03, -4.8828e-03, -4.7913e-03,  ...,  3.0518e-05,
          0.0000e+00,  3.0518e-05]),
 34720,
 'дай бог чтоб просветились мы',
 tensor([ 1, 48, 13,  1, 85, 35, 66, 25, 88,  7, 70, 74, 27, 71,  1,  4, 31]),
 17)

In [214]:
batch_size = 8
num_workers = 0

libri_speech_dev_dl = DataLoader(
    libri_speech_dev_ds, batch_size=batch_size, shuffle=False,
    num_workers=num_workers, pin_memory=False, collate_fn=collate_fn
)
batch = next(iter(libri_speech_dev_dl))
batch

{'audio_path': ['/home/jupyter/mnt/datasets/LibriSpeech_ds/dev/audio/5397/2145/poemi_16_pushkin_0039.wav',
  '/home/jupyter/mnt/datasets/LibriSpeech_ds/dev/audio/5397/2145/poemi_19_pushkin_0090.wav',
  '/home/jupyter/mnt/datasets/LibriSpeech_ds/dev/audio/5397/2145/poemi_31_pushkin_0052.wav',
  '/home/jupyter/mnt/datasets/LibriSpeech_ds/dev/audio/5397/2145/poemi_20_pushkin_0033.wav',
  '/home/jupyter/mnt/datasets/LibriSpeech_ds/dev/audio/5397/2145/poemi_19_pushkin_0055.wav',
  '/home/jupyter/mnt/datasets/LibriSpeech_ds/dev/audio/5397/2145/poemi_13_pushkin_0030.wav',
  '/home/jupyter/mnt/datasets/LibriSpeech_ds/dev/audio/5397/2145/poemi_17_pushkin_0055.wav',
  '/home/jupyter/mnt/datasets/LibriSpeech_ds/dev/audio/5397/2145/poemi_19_pushkin_0103.wav'],
 'audio': tensor([[-4.8218e-03, -4.8828e-03, -4.7913e-03,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0071e-03,  1.0071e-03,  8.8501e-04,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-9.3079e-03,

In [215]:
sample_dataset(batch)

In [216]:
from src.preprocessor import AudioToMelSpectrogramPreprocessor

In [217]:
preprocessor = AudioToMelSpectrogramPreprocessor()

In [218]:
features, feature_lengths = preprocessor(batch['audio'], batch['audio_len'])

/usr/local/lib/python3.8/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


# Implement `src.preprocessor.ConvSubsampling`

In [219]:
import copy

def deterministic_fill(module, seed=64, verbose=False, eps=1e-5):
    module = copy.deepcopy(module)
    
    rng = np.random.default_rng(seed)
     
    for name, param in sorted(module.state_dict().items()):
        if verbose:
            print(name)
        data = torch.tensor(rng.random(param.shape), dtype=param.dtype, device=param.device)
        data = (data - data.mean()) / (data.std() + eps)
        param.data.copy_(data)
        
    return module

def diff_check(left, right, decimals):
    return abs(float(left) - float(right)) < 10 ** (-decimals)

In [220]:
from src.preprocessor import ConvSubsampling

In [221]:
pre_encode = ConvSubsampling(
    feat_in=80, feat_out=176,
    sampling_num=2, conv_channels=176
)

In [222]:
features_enc, feature_enc_lengths = pre_encode(torch.transpose(features, 1, 2), feature_lengths)

In [223]:
pre_encode

ConvSubsampling(
  (conv): Sequential(
    (0): Conv2d(1, 176, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(176, 176, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
  )
  (out): Linear(in_features=3520, out_features=176, bias=True)
)

In [224]:
preprocessor.eval()

rng = np.random.default_rng(42)
rnd_audio = torch.tensor(rng.random([13, 12345]), dtype=torch.float32)
rnd_audio_len = torch.tensor(rng.integers(low=0, high=12345, size=[13]), dtype=torch.long)
rnd_features, rnd_feature_lengths = preprocessor(rnd_audio, rnd_audio_len)

rnd_pre_encode = deterministic_fill(pre_encode, seed=64).eval()
rnd_features_enc, rnd_feature_enc_lengths = rnd_pre_encode(torch.transpose(rnd_features, 1, 2), rnd_feature_lengths)

assert sum(p.numel() for p in rnd_pre_encode.parameters()) == 900416
assert list(rnd_features_enc.shape) == [13, 20, 176]
assert rnd_feature_lengths.cpu().tolist() == [43, 48, 34, 32, 54, 45, 41, 59, 21, 4, 72, 16, 31]
print(rnd_features_enc.abs().mean())
assert diff_check(rnd_features_enc.abs().mean(), 1817.9062, 4)

tensor(1817.9126, grad_fn=<MeanBackward0>)


AssertionError: 

# Implement `src.preprocessor.RelPositionalEncoding`

In [225]:
import math
from src.encoding import RelPositionalEncoding

In [232]:
pos_enc = RelPositionalEncoding(
    d_model=176,
    dropout_rate=0.1,
    max_len=5000,
    xscale=math.sqrt(176),
    dropout_rate_emb=0.0,
)

pos_enc.extend_pe(length=5000, device=torch.device('cpu'))

In [235]:
pos_enc.pe.abs().mean()

tensor(0.6376)

In [234]:
assert list(pos_enc.pe.shape) == [1, 9999, 176]
assert diff_check(pos_enc.pe.abs().mean(), 0.6366, 4)

AssertionError: 

In [240]:
preprocessor.eval()
pre_encode.eval()
pos_enc.eval()

rng = np.random.default_rng(42)
rnd_audio = torch.tensor(rng.random([13, 12345]), dtype=torch.float32)
rnd_audio_len = torch.tensor(rng.integers(low=0, high=12345, size=[13]), dtype=torch.long)
rnd_features, rnd_feature_lengths = preprocessor(rnd_audio, rnd_audio_len)

rnd_pre_encode = deterministic_fill(pre_encode, seed=64).eval()
rnd_features_enc, rnd_feature_enc_lengths = rnd_pre_encode(torch.transpose(rnd_features, 1, 2), rnd_feature_lengths)

rnd_features_emb, rnd_pos_emb = pos_enc(rnd_features_enc)

assert list(rnd_features_emb.shape) == [13, 20, 176]
print(rnd_pos_emb.shape)
assert list(rnd_pos_emb.shape) == [1, 39, 176]
print(rnd_features_emb.abs().mean())
print(rnd_pos_emb.abs().mean())
assert diff_check(rnd_features_emb.abs().mean(), 24117.2559, 4)
assert diff_check(rnd_pos_emb.abs().mean(), 0.5670, 4)

torch.Size([1, 39, 176])
tensor(24117.3398, grad_fn=<MeanBackward0>)
tensor(0.5681)


AssertionError: 

# Implement `src.encoder.ConformerEncoder._create_masks`

In [243]:
from src.encoder import ConformerEncoder

In [244]:
max_length, lengths = 10, torch.tensor([4, 5, 2, 7], dtype=torch.long)
pad_mask, att_mask = ConformerEncoder._create_masks(max_length, lengths, device=torch.device('cpu'))

assert list(pad_mask.shape) == [lengths.shape[0], max_length]
assert list(att_mask.shape) == [lengths.shape[0], max_length, max_length]

assert torch.all(pad_mask.sum(dim=1) == max_length - lengths)
assert torch.all(torch.sum(~att_mask, dim=(1, 2)) == lengths ** 2)

print(pad_mask, att_mask, sep='\n')

NameError: name 'pad_mask' is not defined

In [245]:
from src.attention import RelPositionMultiHeadAttention

In [246]:
attention = RelPositionMultiHeadAttention (
    n_head=3,
    n_feat=9,
    dropout_rate=0.05
)

In [273]:
ten1 = torch.randn(10, 13, 9)
mask = torch.zeros(10, 13, 13)
pos_emb = torch.randn(10, 25, 9)

In [274]:
res = attention(ten1, ten1, ten1, mask, pos_emb)

torch.Size([10, 3, 13, 13])
torch.Size([10, 3, 13, 3])


In [275]:
res.shape

torch.Size([10, 13, 9])